In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (roc_auc_score, precision_recall_curve)
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import warnings
import joblib
import json # Although not directly used for file I/O in this script, it's good practice for config handling
from tqdm import tqdm # For progress tracking

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore")

In [ ]:
# --- PART 1: CONFIGURATION AND DATA LOADING ---

print("\n## PART 1: Configuration and Data Loading")

# --- File and Column Configuration ---
ORIGINAL_DATA_PATH = 'cleaned_merged_ieee_bank_10k_version2.csv'
TARGET_COLUMN = 'isFraud'
DATE_COLUMN_CANDIDATES = ['TransactionDate', 'Date', 'TransactionDT', 'month_year', 'timestamp']
UID_COLUMNS_ORIGINAL = ['guid', 'UserID', 'AccountID']
TRANSACTION_AMOUNT_COL = 'TransactionAmt'

# --- Model and Feature Engineering Configuration ---
N_SPLITS = 5
TOP_N_FEATURES = 100

# --- Chatbot-Specific Configurations ---
CHATBOT_CONFIG = {
    'high_risk_threshold': 0.75,
    'medium_risk_threshold': 0.35,
    'low_risk_threshold': 0.15,
    'confidence_levels': {
        'very_high': 0.90, 'high': 0.75, 'medium': 0.50, 'low': 0.25
    },
    'alert_messages': {
        'very_high': " HIGH FRAUD RISK! We've flagged this transaction for your immediate review. Please confirm if this was you.",
        'high': " Potential Fraud Alert: This transaction shows unusual patterns. Please review your recent activity.",
        'medium': " For Your Review: This transaction has been flagged. If it looks unfamiliar, please contact support.",
        'low': " Transaction appears normal. Logged for monitoring."
    }
}

print(f"Loading dataset from: {ORIGINAL_DATA_PATH}")
try:
    df = pd.read_csv(ORIGINAL_DATA_PATH, dtype={'UserID': str, 'AccountID': str, 'guid': str})
    print(f"Dataset loaded successfully. Shape: {df.shape}")

    actual_date_column = None
    for col in DATE_COLUMN_CANDIDATES:
        if col in df.columns:
            try:
                test_series = pd.to_datetime(df[col].head(100), errors='coerce')
                if not test_series.isna().all():
                    actual_date_column = col
                    break
            except Exception:
                continue

    if actual_date_column is None:
        print("WARNING: No suitable date column found. Generating a dummy 'TransactionDate'.")
        df['TransactionDate'] = pd.date_range('2024-01-01', periods=len(df), freq='H')
        actual_date_column = 'TransactionDate'

    DATE_COLUMN = actual_date_column
    print(f"Using date column: {DATE_COLUMN}")

except FileNotFoundError:
    print(f"ERROR: File '{ORIGINAL_DATA_PATH}' not found. Please ensure the CSV is in the correct directory.")
    exit()


## PART 1: Configuration and Data Loading
Loading dataset from: cleaned_merged_ieee_bank_10k_version2.csv
Dataset loaded successfully. Shape: (10000, 131)
Using date column: TransactionDate


In [ ]:
# --- PART 2: LEAK-FREE DATA PREPROCESSING AND FEATURE ENGINEERING ---

print("\n## PART 2: Leak-Free Data Preprocessing and Feature Engineering")

df_processed = df.copy()

df_processed[DATE_COLUMN] = pd.to_datetime(df_processed[DATE_COLUMN], errors='coerce')
df_processed = df_processed.dropna(subset=[DATE_COLUMN]).sort_values(by=DATE_COLUMN).reset_index(drop=True)
print(f"Sorted data chronologically. Shape: {df_processed.shape}")

imputation_values = {}
numerical_cols = [col for col in df_processed.select_dtypes(include=np.number).columns if col != TARGET_COLUMN]
categorical_cols_obj = [col for col in df_processed.select_dtypes(include=['object']).columns if col not in UID_COLUMNS_ORIGINAL]

for col in numerical_cols:
    if df_processed[col].isnull().any():
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        imputation_values[col] = {'type': 'numerical', 'value': float(median_val)}

for col in categorical_cols_obj:
    if df_processed[col].isnull().any():
        mode_val = df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'UNKNOWN'
        df_processed[col].fillna(mode_val, inplace=True)
        imputation_values[col] = {'type': 'categorical', 'value': mode_val}
print("- Missing value imputation completed.")

def create_leak_free_features(df_input, user_col, amount_col, date_col):
    """
    Creates leak-free expanding and rolling features for a given user column.
    Features are shifted by 1 to ensure they only use past information.
    """
    df_temp = df_input.copy()
    df_temp.sort_values([user_col, date_col], inplace=True)
    df_temp.reset_index(drop=True, inplace=True)

    # Expanding Features
    df_temp[f'{user_col}_count_expanding'] = df_temp.groupby(user_col).cumcount()
    df_temp[f'{user_col}_amount_mean_expanding'] = df_temp.groupby(user_col)[amount_col].expanding().mean().reset_index(level=0, drop=True)
    for col in [f'{user_col}_count_expanding', f'{user_col}_amount_mean_expanding']:
        df_temp[col] = df_temp.groupby(user_col)[col].shift(1)
    df_temp[f'{user_col}_count_expanding'].fillna(0, inplace=True)
    df_temp[f'{user_col}_amount_mean_expanding'].fillna(df_temp[amount_col].mean(), inplace=True)

    # Rolling Features - Corrected and simplified approach
    rolling_windows = ['30min', '1H', '2H', '6H', '12H', '24H', '3D', '7D', '14D', '30D']
    rolling_aggs = ['mean', 'sum', 'std', 'min', 'max', 'median', 'skew', 'kurt']

    for window in tqdm(rolling_windows, desc="Creating Rolling Features"):
        rolling_features = df_temp.groupby(user_col).rolling(window, on=date_col, min_periods=1)[amount_col].agg(rolling_aggs)
        for agg in rolling_aggs:
            df_temp[f'{user_col}_amount_{agg}_{window}'] = rolling_features[agg].values

        rolling_count = df_temp.groupby(user_col).rolling(window, on=date_col, min_periods=1)[amount_col].count()
        df_temp[f'{user_col}_txn_count_{window}'] = rolling_count.values

    # Now, shift ALL rolling features by 1 in one go.
    new_rolling_cols = [col for col in df_temp.columns if any(f'_{window}' in col for window in rolling_windows)]
    for col in new_rolling_cols:
        df_temp[col] = df_temp.groupby(user_col)[col].shift(1)
        df_temp[col].fillna(0, inplace=True)

    # Time Since Last Event Features
    if 'ProductCD' in df_temp.columns:
        df_temp['time_since_last_product_txn'] = df_temp.groupby([user_col, 'ProductCD'])[date_col].diff().dt.total_seconds().fillna(0)
        df_temp['time_since_last_product_txn'] = df_temp.groupby([user_col, 'ProductCD'])['time_since_last_product_txn'].shift(1).fillna(0)
    if 'Merchant' in df_temp.columns:
        df_temp['time_since_last_merchant_txn'] = df_temp.groupby([user_col, 'Merchant'])[date_col].diff().dt.total_seconds().fillna(0)
        df_temp['time_since_last_merchant_txn'] = df_temp.groupby([user_col, 'Merchant'])['time_since_last_merchant_txn'].shift(1).fillna(0)

    # Ratio Features
    df_temp['txn_amt_to_avg_ratio'] = df_temp[amount_col] / (df_temp[f'{user_col}_amount_mean_expanding'] + 1e-6)
    df_temp['txn_amt_to_avg_ratio'].fillna(0, inplace=True)
    for window in ['24H', '7D']:
        max_col = f'{user_col}_amount_max_{window}'
        min_col = f'{user_col}_amount_min_{window}'
        if max_col in df_temp.columns:
            df_temp[f'txn_amt_to_max_ratio_{window}'] = df_temp[amount_col] / (df_temp[max_col] + 1e-6)
            df_temp[f'txn_amt_to_max_ratio_{window}'].fillna(0, inplace=True)
        if min_col in df_temp.columns:
            df_temp[f'txn_amt_to_min_ratio_{window}'] = df_temp[amount_col] / (df_temp[min_col] + 1e-6)
            df_temp[f'txn_amt_to_min_ratio_{window}'].fillna(0, inplace=True)

    df_temp.reset_index(drop=True, inplace=True)
    return df_temp

actual_uid_cols = [col for col in UID_COLUMNS_ORIGINAL if col in df_processed.columns]
if TRANSACTION_AMOUNT_COL in df_processed.columns and actual_uid_cols:
    df_processed = create_leak_free_features(df_processed, user_col=actual_uid_cols[0],
                                             amount_col=TRANSACTION_AMOUNT_COL, date_col=DATE_COLUMN)
    if len(actual_uid_cols) > 1:
        print(f"Note: Using '{actual_uid_cols[0]}' as the primary UID for advanced rolling features. Consider extending for other UIDs if needed.")
    print("- Leak-free aggregation features completed.")
else:
    print("WARNING: Transaction amount column or UID columns not found. Skipping leak-free aggregation features.")

df_processed['hour'] = df_processed[DATE_COLUMN].dt.hour
df_processed['day_of_week'] = df_processed[DATE_COLUMN].dt.dayofweek
df_processed['day_of_month'] = df_processed[DATE_COLUMN].dt.day
print("- Time-based features created.")

if 'ProductCD' in df_processed.columns and 'Merchant' in df_processed.columns:
    df_processed['user_product_interaction'] = df_processed[actual_uid_cols[0]].astype(str) + '_' + df_processed['ProductCD'].astype(str)
    df_processed['user_merchant_interaction'] = df_processed[actual_uid_cols[0]].astype(str) + '_' + df_processed['Merchant'].astype(str)
    print("- Interaction features created.")

label_encoders = {}
all_categorical_cols = categorical_cols_obj + actual_uid_cols
if 'user_product_interaction' in df_processed.columns:
    all_categorical_cols.append('user_product_interaction')
if 'user_merchant_interaction' in df_processed.columns:
    all_categorical_cols.append('user_merchant_interaction')

for col in all_categorical_cols:
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].fillna('UNKNOWN').astype(str)
        le = LabelEncoder()
        le.fit(list(df_processed[col].unique()) + ['UNKNOWN'])
        df_processed[col] = le.transform(df_processed[col])
        label_encoders[col] = le
print("- Categorical features encoded.")

df_processed.fillna(0, inplace=True)
df_processed.replace([np.inf, -np.inf], 0, inplace=True)
print("- Final NaN and Inf values handled.")


## PART 2: Leak-Free Data Preprocessing and Feature Engineering
Sorted data chronologically. Shape: (10000, 131)
- Missing value imputation completed.


Creating Rolling Features: 100%|██████████| 10/10 [00:45<00:00,  4.57s/it]


Note: Using 'guid' as the primary UID for advanced rolling features. Consider extending for other UIDs if needed.
- Leak-free aggregation features completed.
- Time-based features created.
- Interaction features created.
- Categorical features encoded.
- Final NaN and Inf values handled.


In [ ]:
# --- PART 3: FEATURE SELECTION AND PREPARATION ---

print("\n## PART 3: Feature Selection and Preparation")
exclude_cols_final = [DATE_COLUMN, TARGET_COLUMN] + UID_COLUMNS_ORIGINAL
feature_cols = [col for col in df_processed.columns if col not in exclude_cols_final]
X = df_processed[feature_cols].copy()
y = df_processed[TARGET_COLUMN].copy()

selector = VarianceThreshold(threshold=0.001)
X = X[X.columns[selector.fit(X).get_support(indices=True)]]
print(f"- Remaining features after variance threshold: {X.shape[1]}")

final_feature_names = list(X.columns)



## PART 3: Feature Selection and Preparation
- Remaining features after variance threshold: 126


In [ ]:
# --- PART 4: MODEL TRAINING WITH TIME SERIES CROSS-VALIDATION ---

print("\n## PART 4: Model Training with Time Series Cross-Validation")
train_feature_names = [col for col in final_feature_names if col not in UID_COLUMNS_ORIGINAL]
X_train_final = X[train_feature_names]

tscv = TimeSeriesSplit(n_splits=N_SPLITS)
scale_pos_weight = (y == 0).sum() / (y == 1).sum() if (y == 1).sum() > 0 else 1

lgbm_params = {
    'objective': 'binary',
    'metric': 'auc',
    'n_estimators': 3000,
    'learning_rate': 0.005,
    'num_leaves': 60,
    'seed': 42,
    'n_jobs': -1,
    'verbose': -1,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'scale_pos_weight': scale_pos_weight
}

models = {'lgb': []}
oof_predictions = {'lgb': np.zeros(len(X_train_final))}

print("Training LightGBM models...")
for fold, (train_idx, val_idx) in enumerate(tqdm(tscv.split(X_train_final, y), total=N_SPLITS, desc="Training Folds")):
    X_train_fold, X_val_fold = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    lgb_model = lgb.LGBMClassifier(**lgbm_params)
    lgb_model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  eval_metric='auc',
                  callbacks=[lgb.early_stopping(200, verbose=False)])

    val_pred_lgb = lgb_model.predict_proba(X_val_fold)[:, 1]
    oof_predictions['lgb'][val_idx] = val_pred_lgb
    auc_lgb = roc_auc_score(y_val_fold, val_pred_lgb)
    print(f"--- Fold {fold + 1}/{N_SPLITS} AUC: {auc_lgb:.6f} ---")
    models['lgb'].append(lgb_model)

overall_auc = roc_auc_score(y, oof_predictions['lgb'])
print(f"\nOverall Model Performance (OOF AUC): {overall_auc:.6f}")
print("""** How to Interpret the AUC Score **
An AUC of 0.5 is random guessing, and 1.0 is a perfect model.
An AUC between 0.8 and 0.9 (like in the individual folds) is considered 'Excellent'.
The lower Overall OOF AUC suggests the model performs well on stable data but may struggle with concept drift over longer time periods.
This is common in fraud detection as fraud patterns evolve.""")


## PART 4: Model Training with Time Series Cross-Validation
Training LightGBM models...


Training Folds:  20%|██        | 1/5 [00:00<00:02,  1.65it/s]

--- Fold 1/5 AUC: 0.805886 ---


Training Folds:  40%|████      | 2/5 [00:07<00:12,  4.04s/it]

--- Fold 2/5 AUC: 0.937491 ---


...

--- Fold 3/5 AUC: 0.919205 ---


Training Folds:  80%|████████  | 4/5 [00:23<00:06,  6.93s/it]

--- Fold 4/5 AUC: 0.852759 ---


Training Folds: 100%|██████████| 5/5 [00:30<00:00,  6.17s/it]

--- Fold 5/5 AUC: 0.875827 ---

Overall Model Performance (OOF AUC): 0.737187
** How to Interpret the AUC Score **
An AUC of 0.5 is random guessing, and 1.0 is a perfect model.
An AUC between 0.8 and 0.9 (like in the individual folds) is considered 'Excellent'.
The lower Overall OOF AUC suggests the model performs well on stable data but may struggle with concept drift over longer time periods.
This is common in fraud detection as fraud patterns evolve.


In [ ]:
# --- PART 5: CHATBOT-SPECIFIC THRESHOLD OPTIMIZATION ---

print("\n## PART 5: Chatbot-Specific Threshold Optimization")

def optimize_chatbot_thresholds(y_true, y_pred_proba, desired_recall=0.80):
    """
    Optimizes the threshold to achieve a desired recall,
    prioritizing precision at that recall level.
    """
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    idx = np.where(recall >= desired_recall)[0]

    if len(idx) > 0:
        best_idx = idx[np.argmax(precision[idx])]
    else:
        best_idx = np.argmax(recall)

    best_threshold = thresholds[min(best_idx, len(thresholds) - 1)]

    return {'threshold': best_threshold, 'precision': precision[best_idx], 'recall': recall[best_idx]}

oof_mask = oof_predictions['lgb'] != 0
optimal_metrics = optimize_chatbot_thresholds(y[oof_mask], oof_predictions['lgb'][oof_mask], desired_recall=0.80)
CHATBOT_CONFIG['low_risk_threshold'] = optimal_metrics['threshold']

print(f"Optimal Threshold for desired recall of ~80%:")
print(f"  - Threshold: {optimal_metrics['threshold']:.4f}")
print(f"  - Precision: {optimal_metrics['precision']:.4f}")
print(f"  - Recall: {optimal_metrics['recall']:.4f}")
print(f"\nUpdated chatbot low-risk (alert) threshold to {optimal_metrics['threshold']:.4f}")
print("""** How to Interpret Precision and Recall **
- A Recall of {recall:.2f} means: We will successfully catch {recall:.0%} of all actual fraudulent transactions.
- A Precision of {precision:.2f} means: When we do send an alert, there is a {precision:.1%} chance it is for a genuinely fraudulent transaction.
This is a classic trade-off: to catch more fraud (high recall), we must accept more false positives (lower precision).""".format(recall=optimal_metrics['recall'], precision=optimal_metrics['precision']))


## PART 5: Chatbot-Specific Threshold Optimization
Optimal Threshold for desired recall of ~80%:
  - Threshold: 0.0150
  - Precision: 0.0629
  - Recall: 0.8111

Updated chatbot low-risk (alert) threshold to 0.0150
** How to Interpret Precision and Recall **
- A Recall of 0.81 means: We will successfully catch 81% of all actual fraudulent transactions.
- A Precision of 0.06 means: When we do send an alert, there is a 6.3% chance it is for a genuinely fraudulent transaction.
This is a classic trade-off: to catch more fraud (high recall), we must accept more false positives (lower precision).


In [ ]:
# --- PART 6: CHATBOT SIMULATION FUNCTION ---

print("\n## PART 6: Chatbot Simulation Function")

def generate_chatbot_response(probability, config):
    """Generates a human-readable chatbot message based on fraud probability."""
    if probability >= config['confidence_levels']['very_high']:
        return config['alert_messages']['very_high']
    elif probability >= config['confidence_levels']['high']:
        return config['alert_messages']['high']
    elif probability >= config['low_risk_threshold']:
        return config['alert_messages']['medium']
    else:
        return config['alert_messages']['low']

print("- Chatbot response function is ready.")


## PART 6: Chatbot Simulation Function
- Chatbot response function is ready.


In [ ]:
# --- PART 7: DEPLOYMENT ARTIFACTS AND FINAL DEMONSTRATION ---

print("\n## PART 7: Deployment Artifacts and Final Demonstration")

deployment_artifacts = {
    'models': models['lgb'],
    'feature_names': train_feature_names,
    'imputation_values': imputation_values,
    'label_encoders': label_encoders,
    'chatbot_config': CHATBOT_CONFIG,
    'uid_cols': actual_uid_cols
}
joblib.dump(deployment_artifacts, 'fraud_detection_artifacts.pkl')
print("- Deployment artifacts saved to 'fraud_detection_artifacts.pkl'")

print("\n--- FINAL DEMONSTRATION FOR A SINGLE USER ---")

user_id_col_for_demo = 'UserID'
if user_id_col_for_demo not in actual_uid_cols:
    if actual_uid_cols:
        user_id_col_for_demo = actual_uid_cols[0]
    else:
        user_id_col_for_demo = None

if user_id_col_for_demo and user_id_col_for_demo in df_processed.columns and user_id_col_for_demo in label_encoders:
    user_fraud_counts = df_processed.groupby(user_id_col_for_demo)[TARGET_COLUMN].value_counts().unstack(fill_value=0)
    suitable_users_encoded = user_fraud_counts[(user_fraud_counts[1] > 0) & (user_fraud_counts[0] > 0)]

    if not suitable_users_encoded.empty:
        chosen_user_encoded = suitable_users_encoded.index[0]
        user_id_encoder = label_encoders[user_id_col_for_demo]
        chosen_user_original = user_id_encoder.inverse_transform([chosen_user_encoded])[0]

        print(f"Simulating for original user: {chosen_user_original} (Encoded as: {chosen_user_encoded})")
        user_transactions_original = df.loc[df[user_id_col_for_demo] == chosen_user_original].sort_values(by=DATE_COLUMN)

        results = []
        for index, transaction_row in user_transactions_original.iterrows():
            transaction_dict = transaction_row.to_dict()

            for col, details in imputation_values.items():
                if col in transaction_dict and pd.isna(transaction_dict[col]):
                    transaction_dict[col] = details['value']

            for col, le in label_encoders.items():
                if col in transaction_dict:
                    val = transaction_dict[col]
                    transaction_dict[col] = le.transform([str(val) if str(val) in le.classes_ else 'UNKNOWN'])[0]

            temp_df = pd.DataFrame([transaction_dict]).reindex(columns=train_feature_names, fill_value=0)

            probability = np.mean([model.predict_proba(temp_df)[:, 1][0] for model in models['lgb']])

            should_alert = bool(probability >= CHATBOT_CONFIG['low_risk_threshold'])
            chatbot_message = generate_chatbot_response(probability, CHATBOT_CONFIG)

            results.append({
                'Date': pd.to_datetime(transaction_row[DATE_COLUMN]).strftime('%Y-%m-%d'),
                'Amount': f"${transaction_row[TRANSACTION_AMOUNT_COL]:.2f}",
                'ActualFraud': 'Yes' if transaction_row[TARGET_COLUMN] == 1 else 'No',
                'FraudProbability': f"{probability:.1%}",
                'Alert': 'YES' if should_alert else 'NO',
                'ChatbotResponse': chatbot_message
            })

        results_df = pd.DataFrame(results)
        print(results_df.to_string())
    else:
        print("Could not find a user with both fraud and non-fraud transactions for the demo. Please check your data.")
else:
    print(f"The specified user ID column '{user_id_col_for_demo}' was not found or not encoded. Cannot run demo.")

print("\nPIPELINE EXECUTION COMPLETE.")


## PART 7: Deployment Artifacts and Final Demonstration
- Deployment artifacts saved to 'fraud_detection_artifacts.pkl'

--- FINAL DEMONSTRATION FOR A SINGLE USER ---
Simulating for original user: U001 (Encoded as: 0)
           Date    Amount ActualFraud FraudProbability Alert                                                                                       ChatbotResponse
0    2024-01-01    $29.00          No             1.0%    NO                                                    Transaction appears normal. Logged for monitoring.
1    2024-01-01    $39.00          No             0.5%    NO                                                    Transaction appears normal. Logged for monitoring.
2    2024-01-01   $186.00          No             1.5%    NO                                                    Transaction appears normal. Logged for monitoring.
3    2024-01-01    $30.00          No             0.5%    NO                                                    Transaction appea

In [ ]:
!python --version

Python 3.11.13
